In [1]:
import glob
import shutil
import pandas as pd
import numpy as np
import os.path as op
pd.set_option('display.max_columns', 500)

## Load other people's data

### Winnie: computed copy numbers for complexes

``DF_WINNIE_RECIPE``
- Contains recomputed copy numbers in columns ``Copy_number2`` and ``NumUsed`` for complexes

In [2]:
DF_WINNIE_RECIPE = pd.read_csv('./cellpack_P2_prep/MenuWithComplex2.csv', index_col=0)
# Make sure all NAMEs are filled, fill in with Gene ID if empty
for i in DF_WINNIE_RECIPE[pd.isnull(DF_WINNIE_RECIPE.NAME)].index:
    DF_WINNIE_RECIPE.at[i,'NAME'] = DF_WINNIE_RECIPE.at[i, 'Gene']
DF_WINNIE_RECIPE.head(2)

,Gene,UniProt,NAME,Description,Structure_ID,PDB,Structure_source,MW,MOLARITY,Copy_number,Localization,Localization_source,INCLUDE,COLOR,NOTES,model_id,Copy_number2,NumUsed
,,,,,,,,,,,,,,,,,,
0,b0001,P0AD86,thrL,thr operon leader peptide,REP-E00001,E00001_clean-X_clean.pdb,metaTASSER,2138.4426,NaN,NaN,Cytosol,EchoLOCATION predicted,NaN,NaN,NaN,iML1515_v1,NaN,0
1,b0002,P00561,thrA,Bifunctional aspartokinase/homoserine dehydrog...,REP-E00002,E00002_clean-X_clean.pdb,metaTASSER,89119.2202,0.000003,4375.0,Cytosol,iJL1678,x,NaN,NaN,iML1515_v1,4375.0,0


### Eddie: selected structure files for complexes

``EDDIE_CPLX_DICT``
- Contains complex_id -> selected structure files and other stoichiometry info

In [3]:
import json
with open('./cellpack_P2_prep/2018-4-5_final_recipe_1846_enzymes.json', 'r') as f:
    EDDIE_CPLX_DICT = json.load(f)
EDDIE_CPLX_DICT['CYT-O-UBIOX-CPLX_mod_pheme_mod_cu2']

{'GMQE': {'P0ABJ6_72_105_5xtc': 0.13},
 'QMN4': {'P0ABJ6_72_105_5xtc': -2.31},
 'QSPRD': {'P0ABJ6_72_105_5xtc': 0.0},
 'genes': {'b0429': 1, 'b0430': 1, 'b0431': 1, 'b0432': 1},
 'pdbs': {'1fft_bio1_&_NOBFS_0': 1, 'P0ABJ6_72_105_5xtc': 1},
 'quality': 64.01779778870932,
 'use_ECOLI': False,
 'use_PDBs': True,
 'use_SWISS': True,
 'use_homology_other': False}

### Nathan: Eddie's data, parsed

``NATHAN_CPLX_DICT``
- Contains complex id -> selected *single* structure file

``NATHAN_CPLX_DUMMY_DICT``
- Contains complex id -> *multiple* structure files that need to be assembled as a dummy

``NATHAN_CPLX_MISSING_LIST``
- Contains complex ids that were not run through Eddie's pipeline due to various issues

In [4]:
with open('./cellpack_P2_prep/180406-FINAL_FULL_COMPLEXES.json', 'r') as f:
    NATHAN_CPLX_DICT = json.load(f)

with open('./cellpack_P2_prep/180406-FINAL_DUMMY_FULL_COMPLEXES.json', 'r') as f:
    NATHAN_CPLX_DUMMY_DICT = {}
    NATHAN_CPLX_DUMMY_LIST = json.load(f)
    for x in NATHAN_CPLX_DUMMY_LIST:
        NATHAN_CPLX_DUMMY_DICT[x[0]] = x[2]

with open('./cellpack_P2_prep/180406-NATHAN_TO_CHECK.json', 'r') as f:
    NATHAN_CPLX_MISSING_LIST_RAW = json.load(f)
    NATHAN_CPLX_MISSING_LIST = [x[0] for x in NATHAN_CPLX_MISSING_LIST_RAW]
    for x in ['Rph_mono_mod_mg2', 'RNase_P_cplx_mod_2:mg2', 'RNase_P_cplx', 'ribosome']: # Email from Eddie
        if x not in NATHAN_CPLX_MISSING_LIST:
            NATHAN_CPLX_MISSING_LIST.append(x)

## Load EcoliME Complexes & Reactions, create complex --> name dictionary

- Installation
    - Follow instructions for installation at: https://github.com/SBRG/ecolime
- Input
    - E. coli ME model
- Output
    - Complex to complex name dictionary

In [5]:
# FUNCTIONS HERE FROM ECOLIME
import ecolime
import ecolime.flat_files

# First load the list of complexes which tells you complexes + subunit stoichiometry
# Converts the protein_complexes.txt file into a dictionary for ME model construction
_ECOLIME_complexes = ecolime.flat_files.get_complex_subunit_stoichiometry('protein_complexes.txt')

# Then load the modifications which tells you the modificiations (ie. cofactors) that are needed for a complex
# Converts protein_modification.txt
_ECOLIME_complex_modification_dict = ecolime.flat_files.get_complex_modifications('protein_modification.txt', 'protein_complexes.txt')

_ECOLIME_complex_names_df = pd.read_csv(op.join(ecolime.flat_files.ecoli_files_dir, 'protein_complexes.txt'), sep='\t', header=None)
_ECOLIME_complex_id_to_name = {k.strip('#'):v for k,v in _ECOLIME_complex_names_df.set_index(0)[[1]].to_dict()[1].items() if v != 'default_name'}
_ECOLIME_reaction_names_df = pd.read_csv(op.join(ecolime.flat_files.ecoli_files_dir, 'reactions.txt'), sep='\t')
_ECOLIME_reaction_id_to_name = {k.strip('#'):v for k,v in _ECOLIME_reaction_names_df.set_index('#name')[['description']].to_dict()['description'].items()}

# Mapping from complex ID to reaction ID
from collections import defaultdict
import pandas
from os.path import dirname, join, abspath
from ecolime import corrections

def _ECOLIME_fixpath(filename):
    return join(ecolime.flat_files.ecoli_files_dir, filename)

# From: ecolime.flat_files.get_reaction_to_complex, modified to just parse the file
def _ECOLIME_get_complex_to_reaction(modifications=False):
    """anything not in this dict is assumed to be an orphan"""

    complex_to_rxn_dict = defaultdict(set)

    # Load enzyme reaction association dataframe
    df = pandas.read_csv(_ECOLIME_fixpath('enzyme_reaction_association.txt'),
                         delimiter='\t', names=['Reaction', 'Complexes'])
    # Fix legacy naming
    df = df.applymap(lambda x: x.replace('DASH', ''))
    df = df.set_index('Reaction')

    df = corrections.correct_enzyme_reaction_association_frame(df)

    for reaction, complexes in df.itertuples():
        for cplx in complexes.split(' OR '):
            if modifications:
                complex_to_rxn_dict[cplx].add(reaction)
            else:
                complex_to_rxn_dict[cplx.split('_mod_')[0]].add(reaction)

    return complex_to_rxn_dict

#### FUNC: ``get_complex_id``

In [6]:
def get_complex_id(complex_id):
    """Get a complex ID -- helps in converting '_mod_' ones to their core_enzyme"""
    if complex_id in _ECOLIME_complex_modification_dict:
        cplx = _ECOLIME_complex_modification_dict[complex_id]['core_enzyme']
    # These are the normal complexes
    elif complex_id in _ECOLIME_complexes:
        cplx = complex_id
    return cplx

In [7]:
print(get_complex_id('CYTDEAM-MONOMER'))
print(get_complex_id('CYTDEAM-MONOMER_mod_fe2_mod_zn2'))

CYTDEAM-MONOMER
CYTDEAM-MONOMER


#### FUNC: ``get_complex_name``

In [8]:
_ECOLIME_complex_to_reaction = _ECOLIME_get_complex_to_reaction()

def get_complex_name(complex_id):
    """Convert a ME-model complex ID to a human-readable name"""

    complex_id = complex_id.split('_mod_')[0]

    if complex_id in _ECOLIME_complex_id_to_name:
        return _ECOLIME_complex_id_to_name[complex_id]
    # Using reaction name as backup
    elif complex_id in _ECOLIME_complex_to_reaction:
        # Sometimes complex will carry out multiple reactions, just put them all in one big string
        if len(_ECOLIME_complex_to_reaction[complex_id]) == 1:
            rxn = list(_ECOLIME_complex_to_reaction[complex_id])[0]
            return _ECOLIME_reaction_id_to_name[rxn]
        else:
            rxns = list(_ECOLIME_complex_to_reaction[complex_id])
            big_desc = []
            for rxn in rxns:
                try:
                    big_desc.append(_ECOLIME_reaction_id_to_name[rxn])
                except:
                    continue
            return ';'.join(list(set(big_desc)))
    else:
#         print('{}: no name available, returning None'.format(complex_id))
        return complex_id

In [9]:
print(get_complex_name('CYTDEAM-MONOMER'))
print(get_complex_name('CYTDEAM-MONOMER_mod_fe2_mod_zn2'))

Cytosine deaminase
Cytosine deaminase


#### FUNC: ``get_complex_subunits``

In [10]:
def get_complex_subunits(complex_id):
    """Return a list of the complex's subunits"""
    complex_id = get_complex_id(complex_id)
    subunit_list = list(_ECOLIME_complexes[complex_id].keys())
    try:
        subunit_list2 = [k.split('_')[1] for k in subunit_list]
    except:
        raise ValueError('Unknown error parsing stoichiometry for {}, {}'.format(complex_id, list(_ECOLIME_complexes[complex_id].items())))
    return subunit_list2

In [11]:
print(get_complex_subunits('CYTDEAM-MONOMER'))
print(get_complex_subunits('CYTDEAM-MONOMER_mod_fe2_mod_zn2'))

['b0337']
['b0337']


#### FUNC: ``get_complex_stoichiometry_formatted``

In [12]:
def get_complex_stoichiometry_formatted(complex_id):
    """Return a formatted string of the complex's annotated stoichiometry"""
    complex_id = get_complex_id(complex_id)
    
    protein_stoich = ''
    try:
        for k,v in _ECOLIME_complexes[complex_id].items():   # {'protein_b3018': 1.0}
            protein_stoich = protein_stoich + k.split('_')[1] + ':' + str(int(v)) + '; '
    except:
        raise ValueError('Unknown error parsing stoichiometry for {}, {}'.format(complex_id, list(_ECOLIME_complexes[complex_id].items())))
    protein_stoich = protein_stoich.strip('; ')
    
    if protein_stoich:
        return protein_stoich
    else:
        raise ValueError('Unknown error parsing stoichiometry for {}, {}'.format(complex_id, list(_ECOLIME_complexes[complex_id].items())))

In [13]:
print(get_complex_stoichiometry_formatted('CYTDEAM-MONOMER'))
print(get_complex_stoichiometry_formatted('CYTDEAM-MONOMER_mod_fe2_mod_zn2'))

b0337:1
b0337:1


#### FUNC: ``get_complex_localization``

In [14]:
def get_complex_localization(complex_id, localization_df):
    """Return a localization infodict for this complex based on its subunits given a previously made localization df"""
    infodict = {}
    subunits = get_complex_subunits(complex_id)
    all_prev_genes = localization_df[localization_df.Gene.isin(subunits)]
    all_prev_localizations = all_prev_genes['Localization'].unique().tolist()
    all_prev_localization_sources = all_prev_genes['Localization_source'].unique().tolist()
    
    ## If all components of the complex localize to the same place (one unique place) then set that for the complex
    if len(all_prev_localizations) == 1:
        infodict['Localization'] = all_prev_localizations[0]
        infodict['Localization_source'] = all_prev_localization_sources[0]
        infodict['Localization_multiple'] = None
    
    ## OTHERWISE...set localization to the one in the membrane, keep raw information in Localization_multiple
    else:
        if 'Inner_Membrane' in all_prev_localizations and 'Outer_Membrane' in all_prev_localizations:
            raise ValueError('Multiple membrane localizations, please check:', complex_id)
        else:
            for memloc in ['Outer_Membrane', 'Inner_Membrane']:
                if memloc in all_prev_localizations:
                    prev_localizations = dict(localization_df[localization_df.Gene.isin(subunits)].set_index(['Gene', 'Localization']).index.values)
                    infodict['Localization'] = memloc
                    infodict['Localization_source'] = '; '.join(all_prev_localization_sources)
                    infodict['Localization_multiple'] = ''.join([k + ':' + str(v) + '; ' for k,v in prev_localizations.items()]).strip('; ')
    
    return infodict

In [15]:
print(get_complex_localization('CYTDEAM-MONOMER', DF_WINNIE_RECIPE))
print(get_complex_localization('SUCC-DEHASE_mod_3fe4s_mod_fad_mod_2fe2s_mod_4fe4s', DF_WINNIE_RECIPE))

{'Localization': 'Cytosol', 'Localization_source': 'iJL1678', 'Localization_multiple': None}
{'Localization': 'Inner_Membrane', 'Localization_source': 'iJL1678', 'Localization_multiple': 'b0721:Inner_Membrane; b0722:Inner_Membrane; b0723:Cytosol; b0724:Cytosol'}


## Prep final recipe dataframe
- Add complex specific columns
- Add membrane specific columns (to be used in separate cellPACK file later)

In [16]:
DF_FINAL_RECIPE_RAW = DF_WINNIE_RECIPE.copy()

# Complex specific columns
DF_FINAL_RECIPE_RAW['Protein_stoichiometry'] = np.nan
DF_FINAL_RECIPE_RAW['PDB_coverage'] = np.nan
DF_FINAL_RECIPE_RAW['PDB_stoichiometry'] = np.nan
DF_FINAL_RECIPE_RAW['PDB_quality'] = np.nan
DF_FINAL_RECIPE_RAW['Localization_multiple'] = np.nan

# Membrane specific columns
DF_FINAL_RECIPE_RAW['PRINCIPAL_VECTOR'] = np.nan
DF_FINAL_RECIPE_RAW['OFFSET'] = np.nan
DF_FINAL_RECIPE_RAW['JITTER_MAX'] = np.nan
DF_FINAL_RECIPE_RAW['PRINCIPAL_VECTOR_source'] = np.nan
DF_FINAL_RECIPE_RAW['TM1_residues'] = np.nan
DF_FINAL_RECIPE_RAW['TM2_residues'] = np.nan

DF_FINAL_RECIPE_RAW = DF_FINAL_RECIPE_RAW.astype(object)

## Set structure directories

### Final output directory

In [17]:
DIR_FINAL_STRUCTURES = '/home/nathan/projects_unsynced/vizrecon/P2_structures'

### Homology models
NOTE: To reproduce, download homology models at: https://www.dropbox.com/sh/zvnhkk36pxjk3w3/AACho-nxAkSRDgdlpQcjYcEMa?dl=0
- ``sunpro.tar.gz``
- ``zhang.tar.gz``
- ``SWISS-MODEL_renamed``

In [18]:
DIR_SWISS = '/home/nathan/projects_archive/homology_models/ECOLI/SWISS-MODEL_renamed'
DIR_ITASSER = '/home/nathan/projects_archive/homology_models/ECOLI/zhang'
DIR_METATASSER = '/home/nathan/projects_archive/homology_models/ECOLI/sunpro'

### Prototype 1 structure files

In [19]:
DIR_P1_STRUCTURES = '/home/nathan/projects_unsynced/vizrecon/P1_structures'

### Bioassemblies
NOTE: To reproduce, ask Nathan...had to use ssbio code currently in development

In [20]:
DIR_PDB_BIOMOL = '/home/nathan/projects_unsynced/ecoli_gp/data/downloaded_bioassemblies'

#### FUNC: ``get_structure_file``

In [21]:
from ssbio.databases.pdb import PDBProp
def get_structure_file(structure_id):
    """Find a structure file based on a structure ID, return the path"""

    # Check biomols
    if '_bio' in structure_id:
        pdb_bioassembly = op.join(DIR_PDB_BIOMOL, '{}_merged.pdb'.format(structure_id))
        if op.exists(pdb_bioassembly):
            return pdb_bioassembly
        else:
            raise FileNotFoundError('{}: missing bioassembly file, please download and store in {}'.format(structure_id, DIR_PDB_BIOMOL))
    
    # Check prototype 1 files and raw homology dirs
    p1_struct = op.join(DIR_P1_STRUCTURES, '{}.pdb'.format(structure_id))
    p1_struct_gp_swiss = op.join(DIR_P1_STRUCTURES, '{}_clean.pdb'.format(structure_id))
    p1_struct_gp_itasser = op.join(DIR_P1_STRUCTURES, '{}_model1_clean-X_clean.pdb'.format(structure_id))
    p1_struct_gp_sunpro = op.join(DIR_P1_STRUCTURES, '{}_clean-X_clean.pdb'.format(structure_id))
    hom_swiss = op.join(DIR_SWISS, '{}.pdb'.format(structure_id))
    hom_itasser = op.join(DIR_ITASSER, '{}_model1.pdb'.format(structure_id))
    hom_metatasser = op.join(DIR_METATASSER, '{}.pdb'.format(structure_id))
    
    if op.exists(p1_struct):
        return p1_struct
    elif op.exists(p1_struct_gp_swiss):
        return p1_struct_gp_swiss
    elif op.exists(p1_struct_gp_itasser):
        return p1_struct_gp_itasser
    elif op.exists(p1_struct_gp_sunpro):
        return p1_struct_gp_sunpro
    elif op.exists(hom_itasser):
        return hom_itasser
    elif op.exists(hom_swiss):
        return hom_swiss
    elif op.exists(hom_metatasser):
        return hom_metatasser
    elif get_structure_source(structure_id) == 'Protein Data Bank':
        pp = PDBProp(ident=structure_id)
        pp.download_structure_file(outdir=DIR_P1_STRUCTURES, load_header_metadata=False, file_type='pdb')
        return pp.structure_path
    else:
        raise FileNotFoundError('{}: missing file!!'.format(structure_id))

In [22]:
print(get_structure_file('1nen_bio1'))
print(get_structure_file('b0025_1_312_4uze_clean'))
print(get_structure_file('YIDK_ECOLI'))
print(get_structure_file('YHJK_ECOLI'))

/home/nathan/projects_unsynced/ecoli_gp/data/downloaded_bioassemblies/1nen_bio1_merged.pdb
/home/nathan/projects_unsynced/vizrecon/P1_structures/b0025_1_312_4uze_clean.pdb
/home/nathan/projects_archive/homology_models/ECOLI/zhang/YIDK_ECOLI_model1.pdb
/home/nathan/projects_unsynced/vizrecon/P1_structures/YHJK_ECOLI_model1_clean-X_clean.pdb


#### FUNC: ``get_complex_best_structure``

In [23]:
MANUAL_COMPLEX = []

def get_complex_best_structure(complex_id):
    """Return an infodict of the complex GEMbrane results"""
    complex_id = get_complex_id(complex_id)
    
    infodict = {}
    
    infodict['PDB'] = None
    infodict['PDB_coverage'] = None
    infodict['PDB_quality'] = None
    infodict['PDB_stoichiometry'] = None
    infodict['Structure_ID'] = None
    infodict['Structure_source'] = None
    infodict['file_to_copy'] = None
    
    if complex_id in NATHAN_CPLX_DICT:
        structure_id = list(NATHAN_CPLX_DICT[complex_id].keys())[0]
        structure_file = get_structure_file(structure_id)
        infodict['PDB_coverage'] = 'complete'
        infodict['Structure_ID'] = structure_id
        infodict['file_to_copy'] = structure_file
        infodict['PDB'] = op.basename(structure_file)
        if 'bio' in structure_id or len(structure_id) == 4:
            infodict['PDB_quality'] = 'all-experimental'
            infodict['Structure_source'] = 'Protein Data Bank'
        else:
            infodict['PDB_quality'] = 'all-homology'
            if EDDIE_CPLX_DICT[complex_id]['use_ECOLI'] and not EDDIE_CPLX_DICT[complex_id]['use_SWISS'] and not EDDIE_CPLX_DICT[complex_id]['use_homology_other']:
                infodict['Structure_source'] = 'I-TASSER'
            elif not EDDIE_CPLX_DICT[complex_id]['use_ECOLI'] and EDDIE_CPLX_DICT[complex_id]['use_SWISS'] and not EDDIE_CPLX_DICT[complex_id]['use_homology_other']:
                infodict['Structure_source'] = 'SWISS-MODEL'
            elif not EDDIE_CPLX_DICT[complex_id]['use_ECOLI'] and not EDDIE_CPLX_DICT[complex_id]['use_SWISS'] and EDDIE_CPLX_DICT[complex_id]['use_homology_other']:
                infodict['Structure_source'] = 'metaTASSER'
        pdb_stoich = ''
        for k,v in NATHAN_CPLX_DICT[complex_id].items():  # {'1p3h_bio2': 1}
            pdb_stoich = pdb_stoich + k + ':' + str(int(v)) + '; '
        pdb_stoich = pdb_stoich.strip('; ')
        infodict['PDB_stoichiometry'] = pdb_stoich
        
    elif complex_id in NATHAN_CPLX_DUMMY_DICT:
        infodict['PDB_coverage'] = 'dummy'
        
        if complex_id in EDDIE_CPLX_DICT:
            has_exp = False
            has_hom = False
            structure_sources = []
            if EDDIE_CPLX_DICT[complex_id]['use_PDBs']:
                has_exp = True
                structure_sources.append('Protein Data Bank')
            if EDDIE_CPLX_DICT[complex_id]['use_ECOLI']:
                has_hom = True
                structure_sources.append('I-TASSER')
            if EDDIE_CPLX_DICT[complex_id]['use_SWISS']:
                has_hom = True
                structure_sources.append('SWISS-MODEL')
            if EDDIE_CPLX_DICT[complex_id]['use_homology_other']:
                has_hom = True
                structure_sources.append('metaTASSER')
                
            if has_exp and not has_hom:
                infodict['PDB_quality'] = 'all-experimental'
                infodict['Structure_source'] = '; '.join(structure_sources)
            elif has_exp and has_hom:
                infodict['PDB_quality'] = 'mix-experimental-homology'
                infodict['Structure_source'] = '; '.join(structure_sources)
            elif not has_exp and has_hom:
                infodict['PDB_quality'] = 'all-homology'
                infodict['Structure_source'] = '; '.join(structure_sources)
        else:
            raise ValueError('{}: cannot get gembrane info for complex'.format(complex_id))
            
        pdb_stoich = ''
        for k,v in NATHAN_CPLX_DUMMY_DICT[complex_id].items():  # {'1p3h_bio2': 1}
            pdb_stoich = pdb_stoich + k + ':' + str(v) + '; '
        pdb_stoich = pdb_stoich.strip('; ')
        infodict['PDB_stoichiometry'] = pdb_stoich
    
    elif complex_id in NATHAN_CPLX_MISSING_LIST:
        print('[TODO] {}: missing complex'.format(complex_id))
        MANUAL_COMPLEX.append(complex_id)
    else:
        raise ValueError('{}: unknown complex'.format(complex_id))
        
    return infodict

In [24]:
print(get_complex_best_structure('FPPSYN-MONOMER_mod_mn2'))
print(get_complex_best_structure('FABB-CPLX'))
print(get_complex_best_structure('LolCDE-CPLX'))

{'PDB': '1rqj_bio1_merged.pdb', 'PDB_coverage': 'complete', 'PDB_quality': 'all-experimental', 'PDB_stoichiometry': '1rqj_bio1:1', 'Structure_ID': '1rqj_bio1', 'Structure_source': 'Protein Data Bank', 'file_to_copy': '/home/nathan/projects_unsynced/ecoli_gp/data/downloaded_bioassemblies/1rqj_bio1_merged.pdb'}
{'PDB': '2buh_bio2_merged.pdb', 'PDB_coverage': 'complete', 'PDB_quality': 'all-experimental', 'PDB_stoichiometry': '2buh_bio2:1', 'Structure_ID': '2buh_bio2', 'Structure_source': 'Protein Data Bank', 'file_to_copy': '/home/nathan/projects_unsynced/ecoli_gp/data/downloaded_bioassemblies/2buh_bio2_merged.pdb'}
{'PDB': None, 'PDB_coverage': 'dummy', 'PDB_quality': 'all-homology', 'PDB_stoichiometry': 'P0ADC3_6_397_5lj6:1; P75957_3_229_5xu1:1; P75958_52_263_5udf:1', 'Structure_ID': None, 'Structure_source': 'SWISS-MODEL', 'file_to_copy': None}


## Add: complex stoichiometries

### Monomers

In [25]:
monomer_idxes = DF_FINAL_RECIPE_RAW[pd.notnull(DF_FINAL_RECIPE_RAW.Gene)].index
for i in monomer_idxes:
    DF_FINAL_RECIPE_RAW.at[i, 'Protein_stoichiometry'] = DF_FINAL_RECIPE_RAW.at[i, 'Gene'] + ':1'
    
monomer_idxes = DF_FINAL_RECIPE_RAW[(pd.notnull(DF_FINAL_RECIPE_RAW.Gene)) & (pd.notnull(DF_FINAL_RECIPE_RAW.PDB))].index
for i in monomer_idxes:
    DF_FINAL_RECIPE_RAW.at[i, 'PDB_coverage'] = 'complete'
    DF_FINAL_RECIPE_RAW.at[i, 'PDB_stoichiometry'] = DF_FINAL_RECIPE_RAW.at[i, 'PDB'] + ':1'
    if DF_FINAL_RECIPE_RAW.at[i, 'Structure_source'] == 'Protein Data Bank':
        DF_FINAL_RECIPE_RAW.at[i, 'PDB_quality'] = 'all-experimental'
    else:
        DF_FINAL_RECIPE_RAW.at[i, 'PDB_quality'] = 'all-homology'
    shutil.copy(op.join(DIR_P1_STRUCTURES, DF_FINAL_RECIPE_RAW.at[i, 'PDB']), DIR_FINAL_STRUCTURES)

### Complexes

In [26]:
complex_idxes = DF_FINAL_RECIPE_RAW[pd.isnull(DF_FINAL_RECIPE_RAW.Gene)].index

for i in complex_idxes:
    cplx_raw = DF_FINAL_RECIPE_RAW.at[i, 'NAME']

    # Set source (model_id)
    DF_FINAL_RECIPE_RAW.at[i, 'model_id'] = 'ECOLIme'
    
    # Set name/description of complex
    DF_FINAL_RECIPE_RAW.at[i, 'Description'] = get_complex_name(cplx_raw)
    
    # Set annotated stoichiometry
    try:
        DF_FINAL_RECIPE_RAW.at[i, 'Protein_stoichiometry'] = get_complex_stoichiometry_formatted(cplx_raw)
    except ValueError:
        print('{}: cannot store annotated stoichiometry'.format(cplx_raw))
        
    # Set localization
    try:
        localization = get_complex_localization(cplx_raw, DF_WINNIE_RECIPE)
    except ValueError:
        print('{}: cannot store annotated localization'.format(cplx_raw))
    DF_FINAL_RECIPE_RAW.at[i, 'Localization'] = localization['Localization']
    DF_FINAL_RECIPE_RAW.at[i, 'Localization_source'] = localization['Localization_source']
    DF_FINAL_RECIPE_RAW.at[i, 'Localization_multiple'] = localization['Localization_multiple']
    
    # Set coverage by available structure
    best_structures = get_complex_best_structure(cplx_raw)
    DF_FINAL_RECIPE_RAW.at[i, 'PDB'] = best_structures['PDB']
    DF_FINAL_RECIPE_RAW.at[i, 'PDB_coverage'] = best_structures['PDB_coverage']
    DF_FINAL_RECIPE_RAW.at[i, 'PDB_quality'] = best_structures['PDB_quality']
    DF_FINAL_RECIPE_RAW.at[i, 'PDB_stoichiometry'] = best_structures['PDB_stoichiometry']
    DF_FINAL_RECIPE_RAW.at[i, 'Structure_ID'] = best_structures['Structure_ID']
    DF_FINAL_RECIPE_RAW.at[i, 'Structure_source'] = best_structures['Structure_source']
    if best_structures['file_to_copy']:
        shutil.copy(best_structures['file_to_copy'], DIR_FINAL_STRUCTURES)
    elif DF_FINAL_RECIPE_RAW.at[i, 'Copy_number2'] > 0:
        print('{}: need dummy'.format(cplx_raw))

ABC-53-CPLX: need dummy
ACETOLACTSYNIII-CPLX: need dummy
ACETYL-COA-CARBOXYLMULTI-CPLX_mod_btn: need dummy
ANTHRANSYN-CPLX_mod_mg2: need dummy
CARBODEHYDRAT-CPLX_mod_zn2: need dummy
CPLX-157: need dummy
CPLX0-1721: cannot store annotated localization
CPLX0-1923_EG10155-MONOMER: cannot store annotated localization
CPLX0-1923_EG10306-MONOMER: cannot store annotated localization
CPLX0-1924: cannot store annotated localization
CPLX0-1941: cannot store annotated localization
CPLX0-1942: cannot store annotated localization
CPLX0-1943: cannot store annotated localization
CPLX0-2141: cannot store annotated localization
CPLX0-3161_mod_zn2_mod_cobalt2: need dummy
CPLX0-3181: need dummy
CPLX0-3932: cannot store annotated localization
CPLX0-7415: need dummy
CPLX0-7428: need dummy
CPLX0-761: need dummy
CPLX0-7614: need dummy
CPLX0-7719_mod_4fe4s: need dummy
DIAMINOPIMDECARB-CPLX_mod_pydx5p: need dummy
DIHYDROXYACIDDEHYDRAT-CPLX_mod_4fe4s: need dummy
EG11910-MONOMER_dimer_EG11911-MONOMER: cannot sto

## Load membrane data

### Eddie: raw membane protein structure dictionaries

In [27]:
with open('./cellpack_P2_prep/dict_GP1516_OPMNplanes_residues.json', 'r') as f:
    GEMBRANE_OPM_DICT = json.load(f)
with open('./cellpack_P2_prep/dict_GP1516_TMHMMNplanes_residues.json', 'r') as f:
    GEMBRANE_TMHMM_DICT = json.load(f)
with open('./cellpack_P2_prep/dict_GP1516_UniprotNplanes_residues_new.json', 'r') as f:
    GEMBRANE_UNIPROT_DICT = json.load(f)

#### FUNC: ``get_tm_planes``

In [28]:
def get_tm_planes(gene_id, structure_id, gembrane_source):
    infodict = {}
    if gembrane_source == 'OPM':
        infodict['TM1_residues'] = ','.join(GEMBRANE_OPM_DICT[gene_id][structure_id]['OPM_residues_used_to_draw_planes']['TM_1'])
        infodict['TM2_residues'] = ','.join(GEMBRANE_OPM_DICT[gene_id][structure_id]['OPM_residues_used_to_draw_planes']['TM_2'])
    elif 'Uniprot' in gembrane_source:
        keys = gembrane_source.replace('Uniprot ', '').split(' + ')
        if 'TMHMM' in keys:
            keys.remove('TMHMM')
        infodict['TM1_residues'] = ','.join(GEMBRANE_UNIPROT_DICT[gene_id][structure_id]['uniprot_n_planes_residues'][keys[0]])
        infodict['TM2_residues'] = ','.join(GEMBRANE_UNIPROT_DICT[gene_id][structure_id]['uniprot_n_planes_residues'][keys[1]])
    else:
        infodict['TM1_residues'] = ','.join(GEMBRANE_TMHMM_DICT[gene_id][structure_id]['TMHMM_residues_used_to_draw_planes']['tmhmm_leaflet_O'])
        infodict['TM2_residues'] = ','.join(GEMBRANE_TMHMM_DICT[gene_id][structure_id]['TMHMM_residues_used_to_draw_planes']['tmhmm_leaflet_I'])
    return infodict

### Eddie: previously selected membrane protein complexes

In [29]:
DF_GEMBRANE_CPLX = pd.read_csv('./cellpack_P2_prep/eddie_original_membrane_proteins.csv', index_col=0)
DF_GEMBRANE_CPLX.head(2)

,cobraME_complex,type_of_match,gempro_area,pdb_metadata,mw_ratio_GP_to_ME,gene_stoich,Compartment,Reaction
0,ABC-35-CPLX,full_combination,512.782921,"{'CCMA_ECOLI': '2', 'CCMB_ECOLI': '1', 'CCMC_E...",1.112794,"{'b2197': 1, 'b2201': 2, 'b2200': 1, 'b2199': ...",['Inner_Membrane'],protein_b2197_Periplasm + protein_b2198_Inner_...
1,YHIP-MONOMER,full_coverage,1122.254207,{'DTPB_ECOLI': 1},1.162664,{'b3496': 1},['Inner_Membrane'],protein_b3496_Inner_Membrane --> YHIP-MONOMER


### Eddie: previously calculated membrane protein structure qualities

In [30]:
import ast
DF_GEMBRANE_QC = pd.read_csv('./cellpack_P2_prep/df_gp_manually_added_PTS_Cshape_cytoplasmic_adjusted.csv')
DF_GEMBRANE_QC['Gene'] = np.nan
DF_GEMBRANE_QC['Gene_list'] = np.nan
DF_GEMBRANE_QC = DF_GEMBRANE_QC.astype(object)

DF_GEMBRANE_QC['Gene_list'] = DF_GEMBRANE_QC.genes.apply(lambda x: ast.literal_eval(x))
DF_GEMBRANE_QC_monomer_idxs = []
for i,r in DF_GEMBRANE_QC.Gene_list.items():
    if len(r)==1:
        DF_GEMBRANE_QC.at[i, 'Gene'] = r[0]
DF_GEMBRANE_QC = DF_GEMBRANE_QC.rename(columns={'Unnamed: 0':'Structure'})
DF_GEMBRANE_QC.head(2)

,Structure,Average_Area,Average_angle,Average_hydrophobic_thickness,OPM_angle,OPM_area,OPM_distance,OPM_quality,TMHMM_angle,TMHMM_area,TMHMM_distance,TMHMM_quality,Uniprot_angle,Uniprot_area,Uniprot_distance,Uniprot_quality,genes,identical_structures,cobraME,Ecocyc,iJL,Manual_angle,Manual_area,Manual_distance,Manual_quality,Gene,Gene_list
0,FLGG_ECOLI,NaN,NaN,NaN,NaN,"([], [])",NaN,POOR,NaN,"([], [])",NaN,POOR,NaN,"([], [])",NaN,POOR,[u'b1078'],"[u'FLGG_ECOLI', u'5wrh', u'5wrh_bio1']",Present,Present,Absent,0,0,0,0.0,b1078,[b1078]
1,5mdy,NaN,NaN,NaN,NaN,"([], [])",NaN,POOR,NaN,"([], [])",NaN,POOR,NaN,"([], [])",NaN,POOR,[u'b3341'],"[u'5mdy', u'4v48', u'4v47', u'RS7_ECOLI', u'4v...",Present,Present,Present,0,0,0,0.0,b3341,[b3341]


### Rohan: consolidated quality information with chosen vector

In [31]:
# Consolidated vector information for membrane monomers
DF_GEMBRANE_QC_VEC = pd.read_excel('./cellpack_P2_prep/ecoli_membrane_protein_vectors.csv.xlsx')
DF_GEMBRANE_QC_VEC['Gene_list'] = np.nan
DF_GEMBRANE_QC_VEC = DF_GEMBRANE_QC_VEC.astype(object)

DF_GEMBRANE_QC_VEC['Gene_list'] = DF_GEMBRANE_QC_VEC.Gene.apply(lambda x: ast.literal_eval(x))
mem_tovec_monomer_idxs = []
for i,r in DF_GEMBRANE_QC_VEC.Gene_list.items():
    if len(r)==1:
        DF_GEMBRANE_QC_VEC.at[i, 'Gene'] = r[0]
DF_GEMBRANE_QC_VEC.head(2)

,Structure,Gene,Vector,Source,Unit vectors,Gene_list
0,YFIN_ECOLI,b2604,[ 17.52478698 80.60912833 -38.65416456],Average of multiple sources,"[0.19237002502036382, 0.88484841792421154, -0....",[b2604]
1,YAET_ECOLI,b0177,[ 11349.23540503 1744.34042554 5736.72484285],Average of multiple sources,"[0.8841857412896692, 0.13589646149502393, 0.44...",[b0177]


### OPM

#### Premade files
Download OPM files from: http://opm.phar.umich.edu/download.php ("Download all OPM coordinate (.pdb) files as a single .tar file", and untar it)

In [32]:
DIR_OPM = '/home/nathan/projects_unsynced/opm/pdb/'
opm_files = glob.glob(op.join(DIR_OPM, '*.pdb'))
opm_dict = {op.basename(x).replace('.pdb', ''): x for x in opm_files}

#### FUNC: ``run_and_download_opm``

In [33]:
import requests
from ssbio.protein.structure.properties import opm
def run_and_download_opm(orig_file, outdir_html, outdir_opm):
    """Manually run OPM, save the HTML and PDB result"""
#     try:
#     print('{}: running OPM...'.format(orig_file))
    infodict = opm.run_ppm_server(orig_file, outfile=op.join(outdir_html, op.basename(orig_file) + '.opm'))
#     except:
#         print('{}: OPM ERROR'.format(orig_file))
#         return None, None
    newname = op.join(outdir_opm, op.basename(orig_file) + '.opm.pdb')
    
    if not op.exists(newname):
        r = requests.get(infodict['Output file download link'])
        with open(newname, 'wb') as f:
            f.write(r.content)
    
    return newname, infodict

### Rohan: manually changed localizations

In [34]:
manual_memchanger = {'b0068':'Periplasm','b0161':'Periplasm','b0197':'Periplasm','b0632':'Cytosol',
'b0839':'Periplasm','b0878':'Periplasm','b1023':'Periplasm','b1175':'Cytosol','b1452':'Periplasm','b1830':'Periplasm',
'b1847':'Extracellular','b1942':'Cytosol','b2569':'Cytosol','b2793':'Cytosol','b2813':'Periplasm','b2963':'Periplasm',
'b2989':'Cytosol','b3019':'Cytosol'}

## Add: membrane protein information

### Reannotation

In [35]:
reanno_idxes = DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.Gene.isin(list(manual_memchanger.keys()))].index

for i in reanno_idxes:
    DF_FINAL_RECIPE_RAW.at[i, 'Localization'] = manual_memchanger[DF_FINAL_RECIPE_RAW.at[i, 'Gene']]
    DF_FINAL_RECIPE_RAW.at[i, 'Localization_source'] = 'Manual annotation'

#### FUNC: ``get_structure_id``

In [36]:
def get_structure_id(structure_filename):
    """Dumb function to parse a structure filename and return its ID"""
    if 'ECOLI' in structure_filename:
        return structure_filename.split('_model1')[0]
    elif structure_filename.startswith('E'):
        return structure_filename.split('_clean')[0]
    elif structure_filename.startswith('b'):
        return structure_filename.split('_clean')[0]
    elif '-' in structure_filename:
        return structure_filename.split('-')[0]
    elif '_bio' in structure_filename:
        return structure_filename.split('_merged')[0]
    else:
        return structure_filename.split('.pdb')[0]

#### FUNC: ``get_structure_source``

In [37]:
def get_structure_source(structure_id):
    """Dumb function to parse a structure ID and return its source"""
    if 'ECOLI' in structure_id:
        return 'I-TASSER'
    elif '_bio' in structure_id:
        return 'Protein Data Bank'
    elif structure_id.startswith('E'):
        return 'metaTASSER'
    elif len(structure_id) == 4:
        return 'Protein Data Bank'
    else:
#         print('{}: make sure is SWISS-MODEL'.format(structure_id))
        return 'SWISS-MODEL'

#### FUNC: ``get_membrane_info``

In [38]:
def get_membrane_info(structure_filename, gene_id):
    infodict = {}
    infodict['Structure_ID'] = None
    infodict['file_to_copy'] = None
    infodict['PDB'] = None
    infodict['Structure_source'] = None
    infodict['PRINCIPAL_VECTOR'] = None
    infodict['PRINCIPAL_VECTOR_source'] = None
    infodict['TM1_residues'] = None
    infodict['TM2_residues'] = None
    
    structure_id = get_structure_id(structure_filename)
    
    # If contained in OPM database
    if len(structure_id) == 4 and structure_id in opm_dict:
        structure_file = opm_dict[structure_id]
        if not op.exists(structure_file):
            raise FileNotFoundError('{}: cannot find OPM file'.format(structure_file))
        infodict['Structure_ID'] = 'OPM-{}'.format(structure_id)
        infodict['file_to_copy'] = structure_file
        infodict['PDB'] = 'OPM-{}.pdb'.format(structure_id)
        infodict['Structure_source'] = 'OPM; Protein Data Bank'
        infodict['PRINCIPAL_VECTOR'] = '0,0,1'
        infodict['PRINCIPAL_VECTOR_source'] = 'OPM'
        return infodict
    
    checker = DF_GEMBRANE_QC_VEC[DF_GEMBRANE_QC_VEC['Structure'] == structure_id]
    if len(checker) == 0:
        checker = DF_GEMBRANE_QC_VEC[DF_GEMBRANE_QC_VEC['Gene'] == gene_id]        
        
    if len(checker.Structure.values.tolist()) == 1:
        structure_id2 = checker['Structure'].values[0]
        infodict['Structure_ID'] = structure_id2
        gembrane_source = checker['Source'].values[0] 
        structure_file = get_structure_file(structure_id=structure_id2)
        infodict['PDB'] = op.basename(structure_file)
        infodict['file_to_copy'] = structure_file
        infodict['Structure_source'] = get_structure_source(structure_id2)
        infodict.update(get_tm_planes(gene_id=gene_id, structure_id=structure_id2, gembrane_source=gembrane_source))
        infodict['PRINCIPAL_VECTOR'] = checker['Unit vectors'].values[0].strip("'][").replace(' ', '')
        infodict['PRINCIPAL_VECTOR_source'] = 'GEMbrane'
        return infodict
    
    else:
        for x in DF_GEMBRANE_QC[(DF_GEMBRANE_QC.Gene == gene_id) & (DF_GEMBRANE_QC.OPM_quality != 'POOR')].Structure.tolist():
            if structure_id in opm_dict:
                infodict['Structure_ID'] = 'OPM-{}'.format(x)
                structure_file = opm_dict[structure_id]
                infodict['file_to_copy'] = structure_file
                infodict['PDB'] = 'OPM-{}.pdb'.format(structure_id)
                infodict['Structure_source'] = 'OPM; Protein Data Bank'
                infodict['PRINCIPAL_VECTOR'] = '0,0,1'
                infodict['PRINCIPAL_VECTOR_source'] = 'OPM'
                return infodict
            
    # Run OPM manually as a last resort
    pather = get_structure_file(structure_id)
    newname, opm_info = run_and_download_opm(orig_file=pather, 
                                             outdir_html='/home/nathan/projects/vizrecon/opm_manual/opm_html_files', 
                                             outdir_opm='/home/nathan/projects/vizrecon/opm_manual/opm_pdb_files')
    infodict['Structure_ID'] = 'OPM-{}'.format(structure_id)
    infodict['file_to_copy'] = newname
    infodict['PDB'] = 'OPM-{}.pdb'.format(structure_id)
    infodict['Structure_source'] = 'OPM-manual; {}'.format(get_structure_source(structure_id))
    infodict['PRINCIPAL_VECTOR'] = '0,0,1'
    infodict['PRINCIPAL_VECTOR_source'] = 'OPM'
    
    return infodict

In [39]:
get_membrane_info('1qo1-K_clean.pdb', 'b3737')

{'PDB': '1qo1.pdb',
 'PRINCIPAL_VECTOR': '0.79386035541185829,-0.016577879200798225,-0.60787409060311304',
 'PRINCIPAL_VECTOR_source': 'GEMbrane',
 'Structure_ID': '1qo1',
 'Structure_source': 'Protein Data Bank',
 'TM1_residues': 'M_53,T_53,Q_53,O_31,R_31,K_31,P_31,Q_31,N_31,L_31,S_31,L_53,M_31,K_53,T_31,R_53',
 'TM2_residues': 'O_11,P_11,L_73,Q_11,R_11,O_73,T_73,Q_73,K_73,R_73,K_11,M_73,P_73,T_11,M_11,L_11',
 'file_to_copy': '/home/nathan/projects_unsynced/vizrecon/P1_structures/1qo1.pdb'}

In [40]:
print(get_membrane_info('3hfx', 'b0040'))
print(get_membrane_info('E00123', 'b0128'))
print(get_membrane_info('E01548', 'b1641'))

{'Structure_ID': 'OPM-3hfx', 'file_to_copy': '/home/nathan/projects_unsynced/opm/pdb/3hfx.pdb', 'PDB': 'OPM-3hfx.pdb', 'Structure_source': 'OPM; Protein Data Bank', 'PRINCIPAL_VECTOR': '0,0,1', 'PRINCIPAL_VECTOR_source': 'OPM', 'TM1_residues': None, 'TM2_residues': None}
{'Structure_ID': 'YADH_ECOLI', 'file_to_copy': '/home/nathan/projects_archive/homology_models/ECOLI/zhang/YADH_ECOLI_model1.pdb', 'PDB': 'YADH_ECOLI_model1.pdb', 'Structure_source': 'I-TASSER', 'PRINCIPAL_VECTOR': '-0.011198653824636714,-0.047468746665922747,0.99880994600698814', 'PRINCIPAL_VECTOR_source': 'GEMbrane', 'TM1_residues': 'X_169,X_156,X_89,X_227,X_113,X_111,X_31', 'TM2_residues': 'X_244,X_2,X_69,X_193,X_200,X_135'}
{'Structure_ID': 'OPM-E01548', 'file_to_copy': '/home/nathan/projects/vizrecon/opm_manual/opm_pdb_files/E01548_clean-X_clean.pdb.opm.pdb', 'PDB': 'OPM-E01548.pdb', 'Structure_source': 'OPM-manual; metaTASSER', 'PRINCIPAL_VECTOR': '0,0,1', 'PRINCIPAL_VECTOR_source': 'OPM', 'TM1_residues': None, 'T

### Monomers

In [41]:
tmp = DF_FINAL_RECIPE_RAW[pd.notnull(DF_FINAL_RECIPE_RAW.Gene)]
monomer_idxes = tmp[(tmp.Localization.isin(['Inner_Membrane', 'Outer_Membrane'])) & (pd.notnull(tmp.PDB)) & ((tmp.Copy_number2 > 0) | (tmp.Copy_number > 0))].index

for i in monomer_idxes:
    infodict = get_membrane_info(structure_filename=DF_FINAL_RECIPE_RAW.at[i, 'PDB'], gene_id=DF_FINAL_RECIPE_RAW.at[i, 'Gene'])
    for k,v in infodict.items():
        if k == 'file_to_copy':
            continue
        if v:
            DF_FINAL_RECIPE_RAW.at[i, k] = v
    shutil.copy(infodict['file_to_copy'], op.join(DIR_FINAL_STRUCTURES, infodict['PDB']))

### Complexes

In [42]:
DF_GEMBRANE_CPLX.cobraME_complex.tolist()

['ABC-35-CPLX',
 'YHIP-MONOMER',
 'CPLX0-7654',
 'CPLX0-7655',
 'CPLX0-7653',
 'ATPSYN-CPLX_mod_mg2',
 'NARK-MONOMER',
 'CODB-MONOMER',
 'GABP-MONOMER',
 'NANT-MONOMER',
 'GLPT-MONOMER',
 'YGFO-MONOMER',
 'ABC-19-CPLX',
 'TRKA-MONOMER_SAPD-MONOMER_TRKG-MONOMER',
 'YADQ-MONOMER',
 'FORMATEDEHYDROGN-CPLX_mod_bmocogdp',
 'ABC-3-CPLX',
 'YidC_MONOMER',
 'GNTP-MONOMER',
 'FUCP-MONOMER',
 'ABC-26-CPLX',
 'FUMARATE-REDUCTASE_mod_1:pheme_mod_3fe4s_mod_4fe4s_mod_fad_mod_2fe2s',
 'ARAE-MONOMER',
 'ABC-24-CPLX',
 'PHOSPHAGLYPSYN-MONOMER_mod_mg2',
 'NUPC-MONOMER',
 'B2975-MONOMER',
 'ABC-14-CPLX',
 'B0511-MONOMER',
 'B2789-MONOMER',
 'ABC-13-CPLX',
 'CPLX0-1924',
 'SDAC-MONOMER',
 'YFEP-MONOMER',
 'YJCG-MONOMER',
 'CYNX-MONOMER',
 'FORMATEDEHYDROGO-CPLX_mod_bmocogdp',
 'ABC-49-CPLX',
 'CPLX0-3976_edit',
 'CYCA-MONOMER',
 'B1634-MONOMER',
 'ABC-40-CPLX',
 'BRNQ-MONOMER',
 'ABC-15-CPLX',
 'CYT-D-UBIOX-CPLX_mod_pheme_mod_hemed',
 'CYT-O-UBIOX-CPLX_mod_pheme_mod_cu2',
 'KUP-MONOMER',
 'EXUT-MONOMER',


#### FUNC: ``get_complex_membrane_info``

In [43]:
MANUAL_IGNORE = []

def get_complex_membrane_info(complex_id):
    
    infodict = {}
    infodict['Structure_ID'] = None
    infodict['file_to_copy'] = None
    infodict['PDB'] = None
    infodict['Structure_source'] = None
    infodict['PRINCIPAL_VECTOR'] = None
    infodict['PRINCIPAL_VECTOR_source'] = None
    infodict['TM1_residues'] = None
    infodict['TM2_residues'] = None
    
    # Get original selected structure for this membrane complex, check if it is a single structure file
    have_old_cplx = False
    use_old_cplx = False
    complex_id_orig = get_complex_id(complex_id)
    eddie_complexes = DF_GEMBRANE_CPLX.cobraME_complex.tolist()
    if complex_id in eddie_complexes:
        complex_id_final = complex_id
        have_old_cplx = True
    elif complex_id_orig in eddie_complexes:
        complex_id_final = complex_id_orig
        have_old_cplx = True
#     else:
#         print('{}: no previously selected complex structure'.format(complex_id))
#         pass
    if have_old_cplx:
        selected_cplx_row = DF_GEMBRANE_CPLX[DF_GEMBRANE_CPLX.cobraME_complex == complex_id_final]
        selected_cplx = ast.literal_eval(selected_cplx_row.pdb_metadata.values[0])
        if len(selected_cplx) == 1 and int(list(selected_cplx.values())[0]) == 1:
            # Check if old vector exists
            structure_id = list(selected_cplx.keys())[0]
            checker = DF_GEMBRANE_QC_VEC[DF_GEMBRANE_QC_VEC['Structure'] == structure_id]
            if len(checker) > 0:
                use_old_cplx = True
    
    # Get newly selected structure for this membrane complex if old one is not suitable
    if not use_old_cplx:
        selected_cplx_infodict = get_complex_best_structure(complex_id_orig)
        selected_cplx_str = selected_cplx_infodict['PDB_stoichiometry']
        selected_cplx = {x.split(':')[0]:int(x.split(':')[1]) for x in selected_cplx_str.split('; ')}
        
    # Do not use this complex in the recipe if it needs a dummy assembled -- too complicated for membrane proteins
    if len(selected_cplx) > 1:
        print('[TODO] {}: add copy numbers back'.format(complex_id))
        MANUAL_IGNORE.append(complex_id)
        return infodict
    elif len(selected_cplx) == 1 and list(selected_cplx.values())[0] > 1:
        print('[TODO] {}: add copy numbers back'.format(complex_id))
        MANUAL_IGNORE.append(complex_id)
        return infodict
    
#     print('{}: STRUCTURE TO USE:'.format(complex_id), selected_cplx)
    structure_id = list(selected_cplx.keys())[0]
    
    # Check if an OPM structure exists for this selected structure
    if get_structure_source(structure_id) == 'Protein Data Bank' and structure_id in opm_dict:
        structure_file = opm_dict[structure_id]
        if not op.exists(structure_file):
            raise FileNotFoundError('{}: cannot find OPM file'.format(structure_file))
        infodict['Structure_ID'] = 'OPM-{}'.format(structure_id)
        infodict['file_to_copy'] = structure_file
        infodict['PDB'] = 'OPM-{}.pdb'.format(structure_id)
        infodict['Structure_source'] = 'OPM; Protein Data Bank'
        infodict['PRINCIPAL_VECTOR'] = '0,0,1'
        infodict['PRINCIPAL_VECTOR_source'] = 'OPM'
        return infodict
    
    # Check if old vector exists
    if use_old_cplx:
        checker = DF_GEMBRANE_QC_VEC[DF_GEMBRANE_QC_VEC['Structure'] == structure_id]
        structure_file = get_structure_file(structure_id)
#         print('{}: OLD STRUCTURE TO USE:'.format(complex_id), structure_id, structure_file)
        gembrane_source = checker['Source'].values[0] 
        gembrane_gene = get_complex_subunits(complex_id)[0]  # Just pick any gene, should work
        infodict.update(get_tm_planes(gene_id=gembrane_gene, structure_id=structure_id, gembrane_source=gembrane_source))
        infodict['PRINCIPAL_VECTOR'] = checker['Unit vectors'].values[0].strip("'][").replace(' ', '')
        infodict['PRINCIPAL_VECTOR_source'] = 'GEMbrane'
        infodict['Structure_ID'] = structure_id
        infodict['Structure_source'] = get_structure_source(structure_id)
        infodict['file_to_copy'] = structure_file
        infodict['PDB'] = op.basename(structure_file)
        return infodict
    
#     print('{}: NEW STRUCTURE TO USE:'.format(complex_id), structure_id, get_structure_file(structure_id))
     # Run OPM manually as a last resort
    pather = get_structure_file(structure_id)
    newname, opm_info = run_and_download_opm(orig_file=pather, 
                                             outdir_html='/home/nathan/projects/vizrecon/opm_manual/opm_html_files', 
                                             outdir_opm='/home/nathan/projects/vizrecon/opm_manual/opm_pdb_files')
    infodict['Structure_ID'] = 'OPM-{}'.format(structure_id)
    infodict['file_to_copy'] = newname
    infodict['PDB'] = 'OPM-{}.pdb'.format(structure_id)
    infodict['Structure_source'] = 'OPM-manual; {}'.format(get_structure_source(structure_id))
    infodict['PRINCIPAL_VECTOR'] = '0,0,1'
    infodict['PRINCIPAL_VECTOR_source'] = 'OPM'

    return infodict

In [44]:
print(get_complex_membrane_info('ATPSYN-CPLX_mod_mg2'))
print(get_complex_membrane_info('CPLX-157'))

{'Structure_ID': '5t4q', 'file_to_copy': '/home/nathan/projects_unsynced/vizrecon/P1_structures/5t4q.pdb', 'PDB': '5t4q.pdb', 'Structure_source': 'Protein Data Bank', 'PRINCIPAL_VECTOR': '-0.042347511209576708,0.050189026096040812,0.99784154551405901', 'PRINCIPAL_VECTOR_source': 'GEMbrane', 'TM1_residues': 'U_53,N_53,U_31,P_31,Q_31,V_53,V_31,O_31,R_53,O_53,P_53,M_53,S_53,T_53,M_31,R_31,N_31,S_31,T_31,K_262', 'TM2_residues': 'O_11,P_11,U_11,V_11,K_120,K_146,N_73,R_73,O_73,V_73,P_73,M_73,R_11,T_73,Q_73,T_11,U_73,S_73,S_11,Q_11,M_11,N_11'}
[TODO] CPLX-157: add copy numbers back
{'Structure_ID': None, 'file_to_copy': None, 'PDB': None, 'Structure_source': None, 'PRINCIPAL_VECTOR': None, 'PRINCIPAL_VECTOR_source': None, 'TM1_residues': None, 'TM2_residues': None}


In [45]:
tmp = DF_FINAL_RECIPE_RAW[pd.isnull(DF_FINAL_RECIPE_RAW.Gene)]
complex_idxes = tmp[(tmp.Localization.isin(['Inner_Membrane', 'Outer_Membrane'])) & ((tmp.Copy_number2 > 0) | (tmp.Copy_number > 0))].index

for i in complex_idxes:
    complex_id = DF_FINAL_RECIPE_RAW.at[i, 'NAME']
    
    infodict = get_complex_membrane_info(complex_id)
    for k,v in infodict.items():
        if k == 'file_to_copy':
            continue
        if v:
            DF_FINAL_RECIPE_RAW.at[i, k] = v
    if infodict['file_to_copy']:
        shutil.copy(infodict['file_to_copy'], op.join(DIR_FINAL_STRUCTURES, infodict['PDB']))

[TODO] ABC-53-CPLX: add copy numbers back
[TODO] CPLX-157: add copy numbers back
[TODO] G7322-MONOMER_G6561-MONOMER: add copy numbers back
[TODO] LolCDE-CPLX: add copy numbers back
[TODO] Sec-CPLX: add copy numbers back


## Manual stuff to deal with
- Just adding everything back to the original subunits
    - SRP-CPLX could potentially be represented by PDB:5gah but the annotation for the complex here is of the monomer subunit anyways

#### FUNC: ``add_back_cplx_copy_numbers``

In [46]:
def add_back_cplx_copy_numbers(complex_id):
    add_backer = {}
    
    complex_id = get_complex_id(complex_id)
    subunits = {k.split('_')[1]:v for k,v in _ECOLIME_complexes[complex_id].items()}
    tmp = DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.NAME == complex_id]
    if len(tmp)>0:
        num_to_add_back = tmp.Copy_number2.values[0]
        
        for g, mult in subunits.items():
            add_backer[g] = num_to_add_back*mult
            
    return add_backer

In [47]:
for cplx in MANUAL_COMPLEX:
    if cplx in DF_FINAL_RECIPE_RAW.NAME.tolist():
        infodict = add_back_cplx_copy_numbers(cplx)
        for x in DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.Gene.isin(list(infodict.keys()))].index:
            DF_FINAL_RECIPE_RAW.at[x, 'Copy_number2'] = DF_FINAL_RECIPE_RAW.at[x, 'Copy_number2'] + infodict[DF_FINAL_RECIPE_RAW.at[x, 'Gene']]
            DF_FINAL_RECIPE_RAW.at[x, 'NumUsed'] = DF_FINAL_RECIPE_RAW.at[x, 'NumUsed'] - infodict[DF_FINAL_RECIPE_RAW.at[x, 'Gene']]
        rm_idx = DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.NAME == cplx].index[0]
        DF_FINAL_RECIPE_RAW.at[rm_idx, 'INCLUDE'] = np.nan
    
for cplx in MANUAL_IGNORE:
    if cplx in DF_FINAL_RECIPE_RAW.NAME.tolist():
        infodict = add_back_cplx_copy_numbers(cplx)
        for x in DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.Gene.isin(list(infodict.keys()))].index:
            DF_FINAL_RECIPE_RAW.at[x, 'Copy_number2'] = DF_FINAL_RECIPE_RAW.at[x, 'Copy_number2'] + infodict[DF_FINAL_RECIPE_RAW.at[x, 'Gene']]
            DF_FINAL_RECIPE_RAW.at[x, 'NumUsed'] = DF_FINAL_RECIPE_RAW.at[x, 'NumUsed'] - infodict[DF_FINAL_RECIPE_RAW.at[x, 'Gene']]
        rm_idx = DF_FINAL_RECIPE_RAW[DF_FINAL_RECIPE_RAW.NAME == cplx].index[0]
        DF_FINAL_RECIPE_RAW.at[rm_idx, 'INCLUDE'] = np.nan

## Recalculate: MOLARITY

Volumes
- Total cell volume: 2.19955 cubic micrometers
- Cytoplasm: 1.76970 um^3
- Periplasm: 0.42985 um^3

Membrane surface areas:
- Outer membrane: 9.00103e8 angstrom^2
- Inner membrane: 7.73398e8 angstrom^2

In [48]:
import scipy.constants
def convert_copy_number_to_molarity(copies, volume_fl):
    """Convert proteomic copy numbers to a molarity measure.
    
    Args:
        copies (float): Copy number of this specific protein
        volume_fl (float): Volume of solution measured in femtoliters
        
    Returns:
        float: Molarity in mol/liter
        
    """
    volume_l = volume_fl*(10**-15)
    return (copies/scipy.constants.Avogadro)/volume_l

In [49]:
DF_FINAL_RECIPE_RAW['molecules/sqA'] = np.nan

TOTAL_VOL = 2.19955
CYTO_VOL = 1.76970
PERI_VOL = 0.42985
OM_SA = 900103000
IM_SA = 773398000

for i,r in DF_FINAL_RECIPE_RAW.iterrows():
    if r['Localization'] == 'Cytosol':
        DF_FINAL_RECIPE_RAW.loc[i, 'MOLARITY'] = convert_copy_number_to_molarity(r['Copy_number2'], CYTO_VOL)
    elif r['Localization'] == 'Periplasm':
        DF_FINAL_RECIPE_RAW.loc[i, 'MOLARITY'] = convert_copy_number_to_molarity(r['Copy_number2'], PERI_VOL)
    elif r['Localization'] == 'Inner_Membrane':
        DF_FINAL_RECIPE_RAW.loc[i, 'MOLARITY'] = convert_copy_number_to_molarity(r['Copy_number2'], CYTO_VOL)
        DF_FINAL_RECIPE_RAW.loc[i, 'molecules/sqA'] = r['Copy_number2']/IM_SA
    elif r['Localization'] == 'Outer_Membrane':
        DF_FINAL_RECIPE_RAW.loc[i, 'MOLARITY'] = convert_copy_number_to_molarity(r['Copy_number2'], TOTAL_VOL)
        DF_FINAL_RECIPE_RAW.loc[i, 'molecules/sqA'] = r['Copy_number2']/OM_SA

## Mark: INCLUDE

### Full INCLUDE
- Include if:
    - Has PDB
    - Has Copy_number2 > 0

In [50]:
DF_FINAL_RECIPE_FULL = DF_FINAL_RECIPE_RAW.copy()

In [51]:
full_include_idx = DF_FINAL_RECIPE_FULL[(pd.notnull(DF_FINAL_RECIPE_FULL.PDB)) & (DF_FINAL_RECIPE_FULL.Copy_number2>0) & (DF_FINAL_RECIPE_FULL.Localization != 'Extracellular')].index
full_not_include_idx = DF_FINAL_RECIPE_FULL[(pd.isnull(DF_FINAL_RECIPE_FULL.PDB)) | (DF_FINAL_RECIPE_FULL.Copy_number2<=0) | (pd.isnull(DF_FINAL_RECIPE_FULL.Copy_number2)) | (DF_FINAL_RECIPE_FULL.Localization == 'Extracellular')].index

In [52]:
print('Include:', len(full_include_idx))
print('Not include:', len(full_not_include_idx))
len(full_include_idx) + len(full_not_include_idx) == len(DF_FINAL_RECIPE_RAW)

Include: 2136
Not include: 3595


True

In [53]:
for ix in full_include_idx:
    DF_FINAL_RECIPE_FULL.at[ix, 'INCLUDE'] = 'x'
for ix in full_not_include_idx:
    DF_FINAL_RECIPE_FULL.at[ix, 'INCLUDE'] = np.nan

In [54]:
DF_FINAL_RECIPE_FULL.to_csv('./cellpack_P2_prep/180409-RECIPE_FINAL_FULL.csv')

### Not including potential bad quality structures

- Removes anything with "manual" in the `Structure_source` column

In [55]:
DF_FINAL_RECIPE_MIN = DF_FINAL_RECIPE_RAW.copy()

In [56]:
nomanual = [x for x in DF_FINAL_RECIPE_MIN[pd.notnull(DF_FINAL_RECIPE_MIN.Structure_source)].Structure_source.unique().tolist() if 'manual' in x]
min_include_idx = DF_FINAL_RECIPE_MIN[(pd.notnull(DF_FINAL_RECIPE_MIN.PDB)) & (DF_FINAL_RECIPE_MIN.Copy_number2>0) & (DF_FINAL_RECIPE_MIN.Localization != 'Extracellular') & (~DF_FINAL_RECIPE_MIN.Structure_source.isin(nomanual))].index
min_not_include_idx = DF_FINAL_RECIPE_MIN[(pd.isnull(DF_FINAL_RECIPE_MIN.PDB)) | (DF_FINAL_RECIPE_MIN.Copy_number2<=0) | (pd.isnull(DF_FINAL_RECIPE_MIN.Copy_number2)) | (DF_FINAL_RECIPE_MIN.Localization == 'Extracellular') | (DF_FINAL_RECIPE_MIN.Structure_source.isin(nomanual))].index

In [57]:
print('Include:', len(min_include_idx))
print('Not include:', len(min_not_include_idx))
len(min_include_idx) + len(min_not_include_idx) == len(DF_FINAL_RECIPE_MIN)

Include: 1932
Not include: 3799


True

In [58]:
for ix in min_include_idx:
    DF_FINAL_RECIPE_MIN.at[ix, 'INCLUDE'] = 'x'
for ix in min_not_include_idx:
    DF_FINAL_RECIPE_MIN.at[ix, 'INCLUDE'] = np.nan

In [59]:
DF_FINAL_RECIPE_FULL.to_csv('./cellpack_P2_prep/180409-RECIPE_FINAL_MIN.csv')